In [ ]:
import tensorflow as tf
#import tensorflow_quantum as tfq

import cirq
import sympy
import numpy as np
import seaborn as sns
import collections

# visualization tools
%matplotlib inline
import matplotlib.pyplot as plt
from cirq.contrib.svg import SVGCircuit
from PIL import Image

In [ ]:
def convert_to_circuit(image):
    values = np.ndarray.flatten(image)
    qubits = cirq.GridQubit.rect(4, 4)
    circuit = cirq.Circuit()
    for i, value in enumerate(values):
        if value:
            circuit.append(cirq.X(qubits[i]))
    return circuit

In [ ]:
image=np.random.randint(255,size=(2,2))
#image=np.ones(255,size=(2,2))
plt.imshow(image,cmap='gray',vmin=0,vmax=255)
plt.show()
plt.imsave('grayimg.jpeg',image,cmap='gray')

In [ ]:
FILENAME='grayimg.jpeg'
image2 = Image.open(FILENAME)
pixels=np.asarray(image2)
pixels=pixels.astype('float32')
pixels2=pixels/255.0
print(pixels2)
THRESHOLD = 0.5
image_new_bin=np.array(pixels2 > THRESHOLD, dtype=np.float32)
image_new_bin
image_circuit=convert_to_circuit(image_new_bin)

In [ ]:
pix_val = list(image2.getdata())
pix_val
pix_val_flat = [x for sets in pix_val for x in sets]
pix_val_flat

In [ ]:
pixel_mat = np.array(image2.getdata())
width = image2.size[0]

pixel_ind = np.where((pixel_mat[:, :3] ==pix_val_flat[0]).any(axis=1))[0]
coordinate = np.concatenate(
    [
        (pixel_ind % width).reshape(-1, 1),
        (pixel_ind // width).reshape(-1, 1),
    ],
    axis=1,
)
coord=coordinate[0]
#xprime=list(coordinate[[0],coordinate[1])
xprime=coord.tolist()
xprime

In [ ]:
SVGCircuit(image_circuit)

In [ ]:
"""Get qubits to use in the circuit for Grover's algorithm."""
# Number of qubits n.
nqubits = 2

# Get qubit registers.
qubits = cirq.LineQubit.range(nqubits)
ancilla = cirq.NamedQubit("Ancilla")

In [ ]:
def make_oracle(qubits, ancilla, xprime):
    """Implements the function {f(x) = 1 if x == x', f(x) = 0 if x != x'}."""
    # For x' = (1, 1), the oracle is just a Toffoli gate.
    # For a general x', we negate the zero bits and implement a Toffoli.

    # Negate zero bits, if necessary.
    yield (cirq.X(q) for (q, bit) in zip(qubits, xprime) if not bit)

    # Do the Toffoli.
    yield (cirq.TOFFOLI(qubits[0], qubits[1], ancilla))

    # Negate zero bits, if necessary.
    yield (cirq.X(q) for (q, bit) in zip(qubits, xprime) if not bit)

In [ ]:
def grover_iteration(qubits, ancilla, oracle):
    """Performs one round of the Grover iteration."""
    circuit = cirq.Circuit()

    # Create an equal superposition over input qubits.
    circuit.append(cirq.H.on_each(*qubits))

    # Put the output qubit in the |-⟩ state.
    circuit.append([cirq.X(ancilla), cirq.H(ancilla)])

    # Query the oracle.
    circuit.append(oracle)

    # Construct Grover operator.
    circuit.append(cirq.H.on_each(*qubits))
    circuit.append(cirq.X.on_each(*qubits))
    circuit.append(cirq.H.on(qubits[1]))
    circuit.append(cirq.CNOT(qubits[0], qubits[1]))
    circuit.append(cirq.H.on(qubits[1]))
    circuit.append(cirq.X.on_each(*qubits))
    circuit.append(cirq.H.on_each(*qubits))

    # Measure the input register.
    circuit.append(cirq.measure(*qubits, key="result"))

    return circuit

In [ ]:
"""Select a 'marked' bitstring x' at random."""
xprime = [np.random.randint(0, 1) for _ in range(nqubits)]
xprime=list(coordinate)
xprime=coord.tolist()
print(f"Marked bitstring: {xprime}")


In [ ]:
"""Create the circuit for Grover's algorithm."""
# Make oracle (black box)
oracle = make_oracle(qubits, ancilla, xprime)

# Embed the oracle into a quantum circuit implementing Grover's algorithm.
circuit = grover_iteration(qubits, ancilla, oracle)
print("Circuit for Grover's algorithm:")
print(circuit)

In [ ]:
"""Simulate the circuit for Grover's algorithm and check the output."""
# Helper function.
def bitstring(bits):
    return "".join(str(int(b)) for b in bits)

# Sample from the circuit a couple times.
simulator = cirq.Simulator()
result = simulator.run(circuit, repetitions=10)

# Look at the sampled bitstrings.
frequencies = result.histogram(key="result", fold_func=bitstring)
print('Sampled results:\n{}'.format(frequencies))

# Check if we actually found the secret value.
most_common_bitstring = frequencies.most_common(1)[0][0]
print("\nMost common bitstring: {}".format(most_common_bitstring))
print("Found a match? {}".format(most_common_bitstring == bitstring(xprime)))